In [9]:
import time
import csv
import pandas as pd
import argparse
import numpy as np
import random
from multiprocessing import *
import mplleaflet
import matplotlib.pyplot as plt
from collections import deque

from lib.OsrmEngine import *
from lib.Agents import *
from lib.Demand import *

In [10]:
exe_loc = './osrm-backend-5.6.0/build/osrm-routed'
map_loc = './osrm-backend-5.6.0/greater-london-latest.osrm'

osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()

The routing server "http://0.0.0.0:5000" is killed
The routing server "http://0.0.0.0:5000" starts running


In [11]:
olat = 0.089653
olng = 51.373206
dlat = 0.089282
dlng = 51.350675

N = 100
stime = time.time()
for i in range(N):
    o1 = random.uniform(-0.01, 0.01)
    o2 = random.uniform(-0.01, 0.01)
    d1 = random.uniform(-0.01, 0.01)
    d2 = random.uniform(-0.01, 0.01)
    out = osrm.get_routing(olat+o1, olng+o2, dlat+d1, dlng+d2)
etime = time.time()
print("Avarage running time per request: %.3fms" % ((etime-stime)*1000/N) )

Avarage running time per request: 5.561ms


In [18]:
import copy

B_W = 1.5
B_V = 1.0

class Veh(object):
    """ 
    Veh is a class for vehicles
    Attributes:
        id: sequential unique id
        T: system time at current state
        lat: current latitude
        lng: current longitude
        tlat: target (end of route) latitude
        tlng: target (end of route) longitude
        K: capacity
        n: number of passengers on board
        jobs: a list of jobs in the format of (request id, pickup or dropoff, target lat, target lng)
        route: a list of legs
        t: total duration of the route
        d: total distance of the route
        c: total cost (generalized time) of the passegners
    """ 
    def __init__(self, id, K=4, lat=0.089653, lng=51.373206, T=0.0):
        self.id = id
        self.T = T
        self.lat = lat
        self.lng = lng
        self.tlat = lat
        self.tlng = lng
        self.K = K
        self.n = 0
        self.jobs = deque([])
        self.route = deque([])
        self.t = 0.0
        self.d = 0.0
        self.c = 0.0
        
    def get_location(self):
        return (self.lat, self.lng)
    
    def move_to_location(self, lat, lng):
        self.lat = lat
        self.lng = lng
        
    def build_jobs_from_list(self, osrm, jobs):
        self.jobs.clear()
        self.route.clear()
        self.d = 0.0
        self.t = 0.0
        self.tlat = self.lat
        self.tlng = self.lng
        for j in jobs:
            self.add_job(osrm, j)
        self.update_cost()
    
    def add_job(self, osrm, job):
        self.jobs.append( job )
        out = osrm.get_routing(self.tlat, self.tlng, job[2], job[3])
        assert len(out['legs']) == 1
        leg = Leg(out['legs'][0]['distance'], out['legs'][0]['duration'], steps=[])
        for s in range( len(out['legs'][0]['steps']) ):
            step = Step(out['legs'][0]['steps'][s]['distance'],
                        out['legs'][0]['steps'][s]['duration'],
                        out['legs'][0]['steps'][s]['geometry']['coordinates'])
            leg.add_step(step)
        assert len(step.geo) == 2
        assert step.geo[0] == step.geo[1]
        self.add_leg(leg)
        assert len(self.jobs) == len(self.route)

    def add_leg(self, leg):
        self.route.append(leg)
        self.tlat = leg.steps[-1].geo[1][0]
        self.tlng = leg.steps[-1].geo[1][1]
        self.d += leg.d
        self.t += leg.t
        
    def update_cost(self):
        c = 0.0
        t = 0.0
        n = self.n
        for i in range(len(self.jobs)):
            t += self.route[i].t
            c += n * self.route[i].t * B_V
            n += self.jobs[i][1]
            c += t * B_W if self.jobs[i][1] == 1 else 0
        self.c = c
        
    def move_to_time(self, T):
        dT = T - self.T
        assert dT >= 0
        done = []
        if dT == 0 or len(self.jobs) == 0:
            self.T = T
            return done
        while dT > 0 and len(self.jobs) > 0:
            if self.route[0].t < dT:
                dT -= self.route[0].t
                self.T += self.route[0].t
                self.move_to_location(self.jobs[0][2], self.jobs[0][3])
                done.append( (self.jobs[0][0], self.jobs[0][1], self.T) )
                self.n += self.jobs[0][1]
                self.pop_job()
            else:
                while dT > 0 and len(self.route[0].steps) > 0:
                    if self.route[0].steps[0].t < dT:
                        dT -= self.route[0].steps[0].t
                        self.pop_step()
                    else:
                        pct = dT / self.route[0].steps[0].t
                        self.cut_step(pct)
                        self.move_to_location(self.route[0].steps[0].geo[0][0], self.route[0].steps[0].geo[0][1])
                        self.T = T
                        return done
        assert self.n == 0
        assert np.isclose(self.d, 0.0)
        assert np.isclose(self.t, 0.0)
        self.T = T
        self.d = 0.0
        self.t = 0.0
        return done
    
    def pop_job(self):
        self.jobs.popleft()
        self.pop_leg()
        assert len(self.jobs) == len(self.route)
        
    def pop_leg(self):
        leg = self.route.popleft()
        self.d -= leg.d
        self.t -= leg.t
        
    def pop_step(self):
        step = self.route[0].steps.popleft()
        self.t -= step.t
        self.d -= step.d
        self.route[0].t -= step.t
        self.route[0].d -= step.d
        
    def cut_step(self, pct):
        step = self.route[0].steps[0]
        dis = 0
        sega = step.geo[0]
        for segb in step.geo[1:]:
            dis += np.sqrt( (sega[0] - segb[0])**2 + (sega[1] - segb[1])**2)
            sega = segb
        dis_ = 0
        sega = step.geo[0]
        for segb in step.geo[1:]:
            _dis = np.sqrt( (sega[0] - segb[0])**2 + (sega[1] - segb[1])**2)
            dis_ += _dis
            if dis_ / dis > pct:
                break
            sega = segb
        while step.geo[0] != sega:
            step.geo.pop(0)
        _pct = (pct * dis - dis_ + _dis) / _dis
        
        step.geo[0][0] = sega[0] + _pct * (segb[0] - sega[0])
        step.geo[0][1] = sega[1] + _pct * (segb[1] - sega[1])
        
        self.t -= step.t * pct
        self.d -= step.d * pct
        self.route[0].t -= step.t * pct
        self.route[0].d -= step.d * pct
        self.route[0].steps[0].t -= step.t * pct
        self.route[0].steps[0].d -= step.d * pct  
    
    def get_cost_of_jobs(self, osrm, jobs):
        c = 0.0
        t = 0.0
        n = self.n
        olat = self.lat
        olng = self.lng
        for j in jobs:
            dlat = j[2]
            dlng = j[3]
            dt = osrm.get_duration(olat, olng, dlat, dlng)
            t += dt
            c += n * dt * B_V
            n += j[1]
            c += t * B_W if j[1] == 1 else 0
            olat = dlat
            olng = dlng
        return c 
    
    def draw(self):
        plt.plot(self.lat, self.lng, 'b', marker='o')
        for l in range(len(self.route)):
            for s in range(len(self.route[l].steps)):
                step = np.transpose( self.route[l].steps[s].geo )
                if l == 0:
                    plt.plot(step[0], step[1], 'b', linestyle='-')
                else:
                    plt.plot(step[0], step[1], 'b', linestyle='--', dashes=(1, 3))
                        
    def __str__(self):
        str =  "veh %d at (%.7f, %.7f) when t = %.3f; occupancy = %d/%d" % (
            self.id, self.lat, self.lng, self.t, self.n, self.K)
        str += "\n  has %d job(s), distance = %.1f, duration = %.1f，cost = %.1f" % (
            len(self.jobs), self.d, self.t, self.c)
        for j in self.jobs:
            str += "\n    %s: req %d at (%.7f, %.7f)" % ("pickup" if j[1] > 0 else "dropoff", j[0], j[2], j[3])
        return str
    

class Req(object):
    """ 
    Req is a class for requests
    Attributes:
        id: sequential unique id
        t: request time
        olat: origin latitude
        olng: origin longitude
        dlat: destination latitude
        dlng: destination longitude
    """
    def __init__(self, id, t, olat=0.115662, olng=51.374282, dlat=0.089282, dlng=51.350675):
        self.id = id
        self.t = t
        self.olat = olat
        self.olng = olng
        self.dlat = dlat
        self.dlng = dlng
    
    def get_origin(self):
        return (self.olat, self.olng)
    
    def get_destination(self):
        return (self.dlat, self.dlng)
    
    def draw(self):
        plt.plot(self.olat, self.olng, 'r', marker='s')
        plt.plot(self.dlat, self.dlng, 'r', marker='*')
        plt.plot([self.olat, self.dlat], [self.olng, self.dlng], 'r', linestyle='--', dashes=(0.5,1.5))
    
    def __str__(self):
        return "req %d from (%.7f, %.7f) to (%.7f, %.7f) at t = %.3f" % (
            self.id, self.olat, self.olng, self.dlat, self.dlng, self.t)
    

class Model(object):
    """
    Model is the class for the AMoD system
    Attributes:
        T: system time at current state
        D: average arrival interval (sec)
        demand: demand matrix
        V: number of vehicles
        K: capacity of vehicles
        vehs: the list of vehicles
        N: number of requests
        reqs: the list of requests
        queue: requests in the queue
    """ 
    def __init__(self, D, demand, V=2, K=4):
        self.T = 0.0
        self.D = D
        self.demand = demand
        self.V = V
        self.K = K
        self.vehs = []
        for i in range(V):
            self.vehs.append(Veh(i, K=K))
        self.N = 0
        self.reqs = []
        self.queue = deque([])
        
    def generate_request(self):
        dt = self.D * np.random.exponential()
        rand = np.random.rand()
        for d in demand:
            if d[4] > rand:
                req = Req(0 if self.N == 0 else self.reqs[-1].id+1,
                          dt if self.N == 0 else self.reqs[-1].t+dt,
                          d[1], d[0], d[3], d[2])
                break
        self.N += 1
        return req
        
    def generate_requests_to_time(self, T):
        if self.N == 0:
            req = self.generate_request()
            self.reqs.append(req)
        while self.reqs[-1].t <= T:
            req = self.generate_request()
            self.queue.append(self.reqs[-1])
            self.reqs.append(req)
        assert self.N == len(self.reqs)
        
    def dispatch_at_time(self, osrm, T):
        for v in self.vehs:
            v.move_to_time(T)
        self.generate_requests_to_time(T)
        print(self)
        self.assign(osrm)
        for v in self.vehs:
            print(v)
        self.T = T
        
    def assign(self, osrm):
        l = len(self.queue)
        for i in range(l):
            req = self.queue.popleft()
            (v, jobs) = self.insert_heuristics(osrm, req)
            v.build_jobs_from_list(osrm, jobs)
        
    def insert_heuristics(self, osrm, req):
        dc = np.inf
        v_ = None
        jobs_ = None
        for v in self.vehs:
            jobs = list( copy.deepcopy(v.jobs) )
            l = len(jobs)
            c = v.c
            for i in range(l+1):
                for j in range(i+1, l+2):
                    jobs.insert(i, (req.id, 1, req.olat, req.olng) )
                    jobs.insert(j, (req.id, -1, req.dlat, req.dlng) )
                    c_ = v.get_cost_of_jobs(osrm, jobs)
                    if c_ - c < dc:
                        dc = c_ - c
                        v_ = v
                        jobs_ = copy.deepcopy(jobs)
                    jobs.pop(j)
                    jobs.pop(i)
        return (v_, jobs_)  
    
    def draw(self):
        for v in self.vehs:
            v.draw()
        for r in self.reqs[:-1]:
            r.draw()
        
    def __str__(self):
        str = "AMoD system: %d vehicles of capacity %d; %.1f trips/h" % (self.V, self.K, 3600/self.D)
        str += "\n  at t = %.3f, %d requests, in which %d in queue" % ( self.T, self.N-1, len(self.queue) )
        for r in self.queue:
            str += "\n    " + r.__str__()
        return str

In [31]:
model = Model(360, demand, V=2)

In [32]:
for T in range(0,3600,30):
    model.dispatch_at_time(osrm, T)

AMoD system: 2 vehicles of capacity 4; 10.0 trips/h
  at t = 0.000, 0 requests, in which 0 in queue
veh 0 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, duration = 0.0，cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, duration = 0.0，cost = 0.0
AMoD system: 2 vehicles of capacity 4; 10.0 trips/h
  at t = 0.000, 0 requests, in which 0 in queue
veh 0 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, duration = 0.0，cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, duration = 0.0，cost = 0.0
AMoD system: 2 vehicles of capacity 4; 10.0 trips/h
  at t = 30.000, 0 requests, in which 0 in queue
veh 0 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, duration = 0.0，cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 0 job(s), dis

In [33]:
fig = plt.figure(figsize=(16,10))
model.draw()
mplleaflet.display(fig)

In [17]:
for v in model.vehs:
    print(v)

veh 0 at (0.1040702, 51.4112163) when t = 2923.900; occupancy = 1/4
  has 3 job(s), distance = 28365.5, duration = 2923.9，cost = 3978.1
    dropoff: req 3 at (0.0950427, 51.3625064)
    pickup: req 8 at (0.1015402, 51.3547400)
    dropoff: req 8 at (0.1009038, 51.3774429)
veh 1 at (0.1232879, 51.3944586) when t = 5050.900; occupancy = 2/4
  has 8 job(s), distance = 44601.2, duration = 5050.9，cost = 4507.0
    pickup: req 10 at (0.1252152, 51.3955701)
    pickup: req 11 at (0.1229523, 51.3968114)
    dropoff: req 10 at (0.1140902, 51.3871844)
    dropoff: req 7 at (0.1140902, 51.3871844)
    dropoff: req 1 at (0.1004065, 51.3757290)
    dropoff: req 11 at (0.1015619, 51.3777827)
    pickup: req 4 at (0.0628968, 51.3712046)
    dropoff: req 4 at (0.0739150, 51.3585900)
veh 2 at (0.0452234, 51.3999896) when t = 229.900; occupancy = 0/4
  has 0 job(s), distance = 2045.8, duration = 229.9，cost = 588.0
veh 3 at (0.0539134, 51.3381668) when t = 626.700; occupancy = 0/4
  has 2 job(s), distanc

In [34]:
osrm.kill_server()

The routing server "http://0.0.0.0:5000" is killed
